In [1]:

import pandas as pd
import numpy as np
from tqdm import tqdm

from fracdiff import frac_diff_bestd
from monash_data_utils import convert_tsf_to_dataframe, monash_df_to_gluonts_train_datasets
import os


/home/reshawn/miniconda3/envs/cuml/lib/python3.12/site-packages/gluonts/json.py:102: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
X = pd.read_csv('m4_1165_ffd.csv')

In [10]:
def get_weights_ffd(diff_amt, thresh, lim):
    weights = [1.]
    k = 1
    ctr = 0
    while True:
        # compute the next weight
        weights_ = -weights[-1] * (diff_amt - k + 1) / k

        if abs(weights_) < thresh:
            break

        weights.append(weights_)
        k += 1
        ctr += 1
        if ctr == lim - 1:  # if we have reached the size limit, exit the loop
            break
    weights = np.array(weights[::-1]).reshape(-1, 1)
    return weights
def invert_ffd(new_fd_series, original_series, d, thresh, full_original=False):
    # similar as the original invert_fd function,
    # or (the current frac diff value - the 1 less than complete frac diff calculation for this current value) / the current weight
    # and the current weight will always be 1 as the first in that series
    # so the difference is in the 1 less than complete calc, which would be sliced to a window length
    # that window gets decided by the thresh and series length values used
    print(original_series.shape)
    original_series.reset_index(drop=True, inplace=True) # reset index to ensure regular integer index for easier slicing
    print(original_series.shape)
    weights = get_weights_ffd(d, thresh, len(original_series))
    width = len(weights) - 1 # -1 to allow for slicing a window of len(weights)
    print(width)
    new_unfd_series = original_series.copy()
    new_fd_series = new_fd_series.copy()
    if full_original:
        new_fd_series.index += width # shift the index to align with the original series index, i.e. before nans were dropped and the index reset on the frac diffed series

    for i in range(original_series.index[-1]+1, new_fd_series.index[-1]+1): # weirdly taking the last index + 1 instead of len/shape because of the index shift on the new fd series
        # new_unfd_series will have all known values so far up to i-1 as the index of the final inverted series
        # the slice of the weights for the incomplete calc would be the last len(new_unfd_series) weights + 1 to shift back for the current weight we divide by, which is always 1
        new_unfd_series.loc[i] = (new_fd_series.loc[i] - np.dot(weights[:-1].T, new_unfd_series[i-width:i]))[0]
    return new_unfd_series

In [11]:
fd = X['values_fd'].copy()
o = X['values_o'].copy()[:8530]

In [12]:
invert_ffd(fd, o, 0.95, 0.01)

(8530,)
(8530,)
2


0       7324.400000
1       7201.700000
2       7196.400000
3       7177.000000
4       7140.600000
           ...     
8526    8860.100000
8527    8846.900000
8528    8862.100000
8529    8849.800000
8530    8843.972929
Name: values_o, Length: 8531, dtype: float64

In [13]:
fd

0       180.830219
1       169.379713
2       151.535600
3        51.476157
4       209.385943
           ...    
8526    247.116873
8527    220.691292
8528    252.814929
8529    244.047430
8530    226.188054
Name: values_fd, Length: 8531, dtype: float64